# Image segmentation code using Yolo version 8

## 1. Prepraring your images for train YOLOv8 model
To train a custom YOLO v8 model using your own data, you need to prepare your dataset for YOLO by annotating your images using https://www.makesense.ai/. If you already have annotations in COCO format JSON file, then the annotations can be imported to Roboflow for conversion to YOLO format. Otherwise, if you start from scratch, just annotate datasets on Roboflow https://roboflow.com/. You just need to upload your images along with the JSON file, and Roboflow will convert them to any other format, in our case, YOLO v8.


## 2. Install required libraries
Installing ultralytics library. All other libraries should be pre-installed on colab. If you are working on a local system, please make sure you install matplotlib, Pillow, numpy, Seaborn, and roboflow. You may also want to install pandas and other libraries depending on the task.

In [ ]:
# Install the ultralytics package using pip
!pip install ultralytics

In [ ]:
!pip install GPUtil

## 3. Import libraries

In [ ]:
import os
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import Image
import GPUtil
GPUtil.getAvailable()
import torch
print(torch.cuda.is_available())
!nvidia-smi

## 4. Install and prepare Yolo models

By running the next cell, the models with weights will be automatically downloaded and prepared for use.
For a list of pre-trained models, checkout: https://docs.ultralytics.com/models/yolov8/#key-features

In [ ]:
#Instance
model = YOLO('yolov8n-seg.yaml')  # build a new model from YAML
model = YOLO('yolov8n-seg.pt')  # Transfer the weights from a pretrained model (recommended for training)

## 5. Data paths
1- Set paths of your in .yaml file that contains the names of your classes, number of classes and the directories for train, valid, and test datasets, respectively. and use and load it before training.

2- Destination directory for all results.

In [7]:
# path of the YAML file for your data
yaml_file = 'your_yaml_file.yaml'

# Path for destination directory for all results
des_results = 'path_to_save_my_results'

## 6. Load YAML file
Let us load the YAML file.

In [ ]:
# To see how many classes do you have in YAML
import yaml
with open(yaml_file, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
num_classes

## 7. Train your model using your custom dataset

In [9]:
#Define a project --> Destination directory for all results
project = des_results
#Define subdirectory for this specific training
name = "1000_epochs-" #note that if you run the training again, it creates a directory: 1000_epochs-2

In [ ]:
# Train the model
results = model.train(data=yaml_file,
                      project=project,
                      name=name,
                      epochs=1000,
                      patience=0, #Setting patience=0 disables early stopping.
                      batch=4,
                      imgsz=800)

## 8. Inference and evaluation using the trained model
You will get best and last models you can load the best model or the latest from you path.



In [25]:
# for example, we load the best model
my_new_model = YOLO('des_results/weights/best.pt')

Load an image and perform inference (segmentation).

In [ ]:
new_image = 'path for an input image'
new_results = my_new_model.predict(new_image, conf=0.2)  #Adjust conf threshold

The results are stored in a variable 'new_results'. Since we only have one image for segmentation, we will only have one set of results. Therefore, let us work with that one result.

In [19]:
new_result_array = new_results[0].plot()
plt.figure(figsize=(12, 12))
plt.imshow(new_result_array)
plt.show()

## 9. Extracting masks for multi images and save them.


In [ ]:
cmap_custom = plt.cm.colors.ListedColormap(['black', 'white'])
# Specify input and output directories
input_folder = 'path for your images/'
output_folder = 'path to save resulted masks/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image
for image_name in os.listdir(input_folder):
    # Construct image and output paths
    image_path = os.path.join(input_folder, image_name)
    output_path = os.path.join(output_folder, f'{os.path.splitext(image_name)[0]}_mask.png')

    # Predict mask using YOLO model
    results = my_new_model.predict(image_path, conf=0.2)
    new_result = results[0]
    extracted_mask = new_result.masks.data.cpu().numpy()[0]  # Assuming there is only one mask per image
    # plt.imshow(extracted_mask)
    # plt.show()
    # Save the mask as a black and white image
    #save_mask_as_bw_image(extracted_mask, output_path)
    plt.imsave(output_path, extracted_mask, cmap=cmap_custom)

print("Mask prediction and saving completed.")